In [12]:
import pandas as pd
import re
import os
import json
from spacy.lang.en import English # updated
import spacy
from collections import Counter
from tqdm.notebook import tqdm

In [17]:
df = pd.read_csv('/shared/0/projects/news-quotes/reg_data_plot.csv')

In [4]:
df.columns

Index(['url', 'doi', 'author_id', 'affiliation_ids', 'author_seq_num',
       'author_name', 'author_pos_cate', 'author_last_name',
       'last_name_length', 'last_name_prob', 'author_eth_ethnea',
       'author_gender_ethnea', 'author_eth_wiki', 'author_eth_census',
       'reporter_name', 'reporter_eth_ethnea', 'reporter_gender_ethnea',
       'reporter_eth_wiki', 'reporter_eth_census', 'author_eth_ethnea_broad',
       'reporter_eth_ethnea_broad', 'author_eth_wiki_broad',
       'reporter_eth_wiki_broad', 'is_top_author', 'doi_men_cn', 'author_rank',
       'affiliation_cate', 'affiliation_rank', 'journal_title',
       'journal_impact', 'top_journal', 'num_authors', 'outlet', 'category',
       'num_words', 'num_mentioned_papers', 'mention_year',
       'mention_year_center', 'gap_in_years', 'is_author_mentioned',
       'FleschReadingEase', 'sentences_per_paragraph', 'type_token_ratio',
       'Publication Date', 'Mention Date', 'is_corresponding'],
      dtype='object')

In [5]:
urls_text = {}

with open('/shared/0/projects/news-quotes/crawl_news/url_text_clean.json', 'r') as ofile:
    for row in ofile:
        row = json.loads(row)
        url, text = row['url'], row['text']
        urls_text[url] = text
print(len(urls_text))

520061


In [13]:
nlp = spacy.load('en_core_web_sm')

In [8]:
text = urls_text[list(urls_text.keys())[2]]

In [9]:
text

'New research in The FASEB Journal suggests that differences in DNA methylation patterns and long-term variation in gene activity established in the womb underpins the link between birth size and disease risk\n\nA new research report published in the November 2014 issue of The FASEB Journal suggests that being overweight might be better in the long term than being underweight. Before you reach for that box of Twinkies, however, it\'s important to note that this discovery only applies to the weight of newborn babies in relation to risk of future disease.\n\n"These findings support the hypothesis that common long-term variation in the activity of genes established in the womb may underpin links between size at birth and risk for adult disease," said Claire R. Quilter, Ph.D., study author from the Mammalian Molecular Genetics Group, Department of Pathology at the University of Cambridge in the United Kingdom. "If confirmed these could be important markers of optimal fetal growth and may b

In [10]:
speech_verbs = set([
    'describe', 'explain', 'say', 'tell', 'note', 'add', 'acknowledge', 
    'offer', 'point', 'caution', 'advise', 'emphasize', 'see',
    'suggest', 'comment', 'continue', 'confirm',
    'accord', # "blah," according to LastName
])

verb_counts = Counter()

def check_name(token, sent, last_name_tokens):
    ti = token.i - sent.start
    # print(last_name_tokens)
    found = True
    for i in range(len(last_name_tokens)):
        if sent[ti+i].text != last_name_tokens[i]:
            found = False
            break
    return found
    
def has_quote(text, fa_last_name):
    #print("LAST NAME: '%s'" % fa_last_name)
    sentences = [i for i in nlp(text).sents]
    
    # See if we can find a "said"-like verb to indicate they gave the quote
    is_their_quote = False
    is_their_quote_weaker = False
    
    for sent in sentences:
        
        if is_their_quote or is_their_quote_weaker:
            break
        
        sent_text = str(sent)
        #     if fa_last_name in sent:
        if re.search(r'\b'+fa_last_name +r'\b', sent_text) \
           and ('"' in sent_text): # or "'" in sent_text):           

            if '-' in fa_last_name:
                last_name_tokens = []
                for t in fa_last_name.split('-'):
                    last_name_tokens.append(t)
                    last_name_tokens.append('-')
                last_name_tokens =last_name_tokens[:-1]
                hypenated_last_name = True
            else:
                hypenated_last_name = False
                
            for token in sent:
                # print(token.text)
                if fa_last_name == token.text or (hypenated_last_name and check_name(token, sent, last_name_tokens)):
                    #print("match")
                    ti = token.i - sent.start
                    mn = max(0, ti - 5)
                    mx = min(ti+6, len(sent))
                    #print(ti, mn, mx)
                    
                    for i in range(mn, mx):
                        #print(i, sent[i])
                        if sent[i].pos_ == 'VERB' and sent[i].lemma_ in speech_verbs:
                            is_their_quote = True
                            break
                    
                        if sent[i].pos_ == 'VERB':
                            # print(sent[i].text, sent[i].pos_)
                            verb_counts[sent[i].lemma_] += 1
                                                        
                    for i in range(ti, 0, -1):
                        #print(i, sent[i])
                        if sent[i].lemma_ == '"':
                            break
                        if sent[i].pos_ == 'VERB' and sent[i].lemma_ in speech_verbs:
                            is_their_quote_weaker = True
                            break
                    for i in range(ti, len(sent)):
                        #print(i, sent[i], sent[i].pos_)
                        if sent[i].lemma_ == '"':
                            break
                        if sent[i].pos_ == 'VERB' and sent[i].lemma_ in speech_verbs:
                            is_their_quote_weaker = True
                            break
                      
            if is_their_quote:
                #print("Found a quote by %s:\n  %s\n" % (fa_last_name, " ".join(str(sent_text).split())))
                pass
            elif is_their_quote_weaker:
                #print("Found a POSSIBLE quote by %s:\n  %s\n" % (fa_last_name, " ".join(str(sent_text).split())))
                pass            
            else:
                #print("NO QUOTE by %s:\n  %s\n" % (fa_last_name, " ".join(str(sent_text).split())))
                pass
        #print('')    #break

    return is_their_quote or is_their_quote_weaker

if False:
    for ri, row in df.iterrows():
        url = row['url']
        text = urls_text[url]
        fa_last_name = row['first_aut_last_name']
        has_quote(text, fa_last_name)
        if ri % 100 == 0:
            print(ri, verb_counts.most_common(50), '\n')
        

In [14]:
tests = [
    ('Mongraw-Chaffin', '''Common medical wisdom has been that some people who are obese seemed to be pretty healthy and free from heart disease risks, so they haven't been advised to lose weight or take other steps to prevent future heart disease," said Morgana Mongraw-Chaffin, Ph.D., assistant professor of public health sciences at Wake Forest Baptist and lead author of the study.'''),
    ("Mongraw-Chaffin", '''"In this paper, we specifically looked to see whether that progression was associated with a higher risk for heart disease and we found that it was," Mongraw-Chaffin said.'''),
    ("Montero-Serra", '''The results reveal that sardines and other fish (with fast life cycles, planktonic larval stage and low habitat dependence) are highly vulnerable to changes in ocean temperature, and therefore represent "an exceptional bioindicator to measure the direction and speed of climate change expected in the near future," points out Montero-Serra.'''),
    ("Castles", 'Building it requires good instruction, solid foundations in vocabulary and language comprehension, and extensive reading practice," Castles, Rastle, and Nation note.'),    
]

for fa_last_name, text in tests:
    print(has_quote(text, fa_last_name))


True
True
True
False


In [ ]:
with open('/shared/0/projects/news-quotes/crawl_news/has_author_quotes.tsv', 'wt') as outf:
    outf.write('url\thas_author_quote\n')
    for ri, row in tqdm(df.iterrows(), total=len(df)):
        url = row['url']
        text = urls_text[url]
        fa_last_name = row['author_last_name']
        hq = has_quote(text, fa_last_name)
        #if ri % 100 == 0:
        #    print(ri, verb_counts.most_common(50), '\n')
        outf.write(url + '\t' + str(hq) + '\n')